Se emplean funciones de data engineer, data analitics para realizar y obtener metricas y KPIs

In [ ]:
# Se importan librerías a usar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Se descargan archivos csv a asociados al análisis de datos
Acceso_internet_fijo_velocidadbajada_provincia=pd.read_csv('Datasets/AccesosaInternetfijoporvelocidadbajadayprovincia_2791741685050936567.csv')
Acceso_internet_fijo_velocidadbajada_localidad=pd.read_csv('Datasets/AccesosaInternetfijoporvelocidaddebajadaylocalidad_2776171685050986091.csv')
Conectividad_Servicio_Internet=pd.read_csv('Datasets/ConectividadalserviciodeInternet_2776151685051230707.csv')
Internet_Accesos_por_tecnologia=pd.read_csv('Datasets/Internet_Accesos-por-tecnologia.csv')
Internet_Accesos_velocidad=pd.read_csv('Datasets/Internet_Accesos-por-tecnologia.csv')
Internet_BAF=pd.read_csv('Datasets/Internet_BAF.csv')
Internet_ingresos = pd.read_csv('Datasets/Internet_Ingresos.csv', encoding='utf-8')
Internet_Penetracion = pd.read_csv('Datasets\Internet_Penetracion.csv', encoding='utf-8')
Listado_localidades_conectividad_internet=pd.read_csv('Datasets/Listadodelocalidadesconconectividadainternet_2750221685051211461.csv')
velocidadInternet_anio = pd.read_csv('Datasets\historico_velocidad_internet.csv', encoding='utf-8')

Se considera para el análisis económico primeramente el DF asociado a Acceso_internet_fijo_velocidadbajada_provincia.
Debido a que la velocidad de internet es una medida indiecta de acceso a internet en el país y con ello de crecimiento 
económico asociado desde el punto de vista empresarial. 

In [ ]:
#Evalúo el tipo de datos para cada columna con la finalidad de saber que debo normalizar
velocidadInternet_anio.dtypes

In [ ]:
velocidad_anio = velocidadInternet_anio

In [ ]:
serie_velocidad = velocidad_anio['Mbps (Media de bajada)'].str.replace(',', '.').astype(float)
velocidad_anio = velocidadInternet_anio.drop(columns='Mbps (Media de bajada)',axis=1)
velocidad_anio = pd.concat([velocidad_anio, serie_velocidad], axis=1)
velocidad_anio = velocidad_anio.drop(columns='Trimestre.1',axis=1)

In [ ]:
velocidad_maxima = velocidad_anio['Mbps (Media de bajada)'].max()
velocidad_maxima

En base a estos datos procedemos a sacar el KPI, para lo cual debemos obtener el promedio de crecimiento entre cada una de las provincias y luego 
a este crecimiento sacarle el promedio. Nuestro resultado demuestra que existe un crecimiento promedio de velocidad de internet entre los años
2017 a 2022 de 459% es decir que podemos estimar que para el año 2023 la tasa de velocidad aumentaría posiblemente un 122% 

In [ ]:
crecimiento_total = ((velocidad_anio['Mbps (Media de bajada)'].max() - velocidad_anio['Mbps (Media de bajada)'].min())/ velocidad_anio['Mbps (Media de bajada)'].min())*100
round(crecimiento_total,1)
#crecimiento total a lo largo de los anios

HAY QUE PREDECIR EL CUARTO TRIMESTRE DE 2022, USANDO SOLO LOS CUARTOS TRIMESTRES DE LOS ANIOS PREVIOS A 2022, CREAR UN DATAFRAME NUEVO SOLO CON LOS CUARTOS TRIMESTRES Y LUEGO DE PREDECIRLO AGREGARLO COMO UN REGISTRO NUEVO A "velocidad_anio".

In [ ]:
velocidad_anio2 = velocidad_anio.drop(columns='Trimestre',axis=1)

In [ ]:
velocidad_anio2 = velocidad_anio2.groupby('Año').mean()

In [ ]:
velocidad_anio2=velocidad_anio2.reset_index()

In [ ]:
velocidad_anio2

In [ ]:
x = velocidad_anio2['Año']
y = velocidad_anio2['Mbps (Media de bajada)']
anio = 2023

plt.figure()

#Ajuste de polinomio de grado 4
coef = np.polyfit(x, y, 4)
p = np.polyval(coef, x)

#Predicción para el año 2023
p_pred = np.polyval(coef, anio)

plt.plot(x, p, label='Ajuste polinomial')

mse = np.mean((p - y) ** 2)
print(f'Error cuadrático medio para el polinomio de grado 4: {mse}')

plt.scatter(x, y, color='red', label='Datos reales')
plt.scatter(anio, p_pred, color='green', label=f'Predicción para {anio}')
plt.axvline(anio, color='green', linestyle='--', label=f'Valor predicho para {anio}: {p_pred:.2f}')
plt.xlabel('Año')
plt.ylabel('Mbps (Media de bajada)')
plt.title('Ajuste polinomial a datos')
plt.legend()
plt.show()

In [ ]:
x = velocidad_anio2['Año']
y = velocidad_anio2['Mbps (Media de bajada)']
anio = 2023
coef = np.polyfit(x, y, 4)
p = np.polyval(coef, anio)
print(f'Para el anio {anio} la prediccion es {p}')

Obtengo el análisis del siguiente DF Conectividad_Servicio_Internet, dado que puede arrojar 
datos del tipo de internet por provincia y se pueden sacar metricas de allí y saber cual es 
el tipo de internet predominante en Argentina.

In [ ]:
#Reviso mis datos, lo primero que noto es que en Provincias está todo en mayúscula y no es agradable a la vista, lo decido normalizar
Conectividad_Servicio_Internet.head(5)


In [ ]:
# Convertir la primera letra en mayúscula y el resto en minúscula
Conectividad_Servicio_Internet['Provincia']= Conectividad_Servicio_Internet['Provincia'].str.lower().str.capitalize()
Conectividad_Servicio_Internet

In [ ]:
# Seleccionar las columnas relevantes
columnas_relevantes = ["Provincia", "Localidad", "ADSL", "CABLEMODEM", "DIALUP", "FIBRAOPTICA", "SATELITAL", "WIRELESS", "TELEFONIAFIJA", "3G", "4G"]

# Crear un nuevo DataFrame con las columnas seleccionadas
df_reducido = Conectividad_Servicio_Internet.reindex(columns=columnas_relevantes)

# Utilizar la función melt para transformar las columnas en filas
df_melted = df_reducido.melt(id_vars=["Provincia", "Localidad"], var_name="Tipo de Internet", value_name="Disponible")

# Filtrar los casos donde la conexión está disponible
df_disponible = df_melted[df_melted["Disponible"] != "--"]

# Imprimir el DataFrame resultante
df_disponible


In [ ]:
#Se continúa la normalización de datos 
import re #importo regex

# Definir los patrones de regex para los valores que deseas normalizar
patrones = r"CABLEMODEM|DIALUP|FIBRAOPTICA|SATELITAL|WIRELESS|TELEFONIAFIJA"

# Filtrar los valores que coinciden con los patrones utilizando regex
mascara = df_disponible['Tipo de Internet'].str.contains(patrones, flags=re.IGNORECASE, regex=True)

# Crear una copia del DataFrame para trabajar directamente en el original
df_disponible = df_disponible.copy()

# Aplicar la normalización solo a los valores filtrados
df_disponible.loc[mascara, 'Tipo de Internet'] = df_disponible.loc[mascara, 'Tipo de Internet'].str.lower().str.capitalize()
df_disponible


In [ ]:
# Obtener el conjunto único de provincias
provincias_unicas = df_disponible['Provincia'].unique()

# Obtener el conjunto único de tipos de internet
tipos_internet_unicos = df_disponible['Tipo de Internet'].unique()

# Agrupar por provincia y tipo de internet para contar la distribución
distribucion = df_disponible.groupby(['Provincia', 'Tipo de Internet']).size().reset_index(name='Cantidad')

# Crear la gráfica de barras
plt.figure(figsize=(10, 6))

# Iterar sobre cada tipo de internet
for i, tipo_internet in enumerate(tipos_internet_unicos):
    tipo_internet_provincias = distribucion[distribucion['Tipo de Internet'] == tipo_internet]
    cantidades = tipo_internet_provincias['Cantidad'].values
    
    # Crear las posiciones de las barras para cada tipo de internet
    x = [i] * len(cantidades)
    
    # Graficar las barras
    plt.bar(x, cantidades, label=tipo_internet)
    
# Configurar el eje x
plt.xticks(range(len(tipos_internet_unicos)), tipos_internet_unicos)
plt.xlabel('Tipo de Internet')

# Configurar el eje y
plt.ylabel('Cantidad')
plt.title('Distribución de Tipos de Internet entre las Provincias')
plt.legend()

# Mostrar la gráfica
plt.show()

#Conclusion: En la gráfica se muestra la distribución del tipo de internet en relación a las diferentes provincias observando una
#predomininancia de la telefonía móvil 4G

In [ ]:
#Obtengo el valor de tipo de internet con provincia y grafico 
df_provincia_tipo_internet = df_disponible.groupby('Provincia')['Tipo de Internet'].value_counts()

df_provincia_tipo_internet

Obtenemos metricas que indiquen la tasa de crecimiento en ganancia por miles de pesos argentino para ello
usamos el DF Ingresos por Miles

In [ ]:
#Se convierten la columna de Ingresos (miles de pesos)
Internet_ingresos['Ingresos (miles de pesos)'] = Internet_ingresos['Ingresos (miles de pesos)'].str.replace('.', '').str.replace(',', '').astype(int)

In [ ]:
#Se agrupa por año e Ingresos miles de pesos
Internet_ingresos = Internet_ingresos.groupby('Año').sum()
Internet_ingresos.drop(['Trimestre', 'Periodo'], axis=1, inplace=True)

In [ ]:
Internet_ingresos.reset_index()

In [ ]:
# Calculamos la tasa de crecimiento en porcentaje
Internet_ingresos['Crecimiento_anual'] = Internet_ingresos['Ingresos (miles de pesos)'].pct_change() * 100

# Replace NaN values with 0 for the year 2014
Internet_ingresos.loc[Internet_ingresos['Año'] == 2014, 'Crecimiento_anual'] = 0


In [ ]:
#Realizo un reindex de columna
Internet_ingresos = Internet_ingresos.reset_index()

In [ ]:
#Garantizo que si quedaran
Internet_ingresos.columns

In [ ]:
# Filtrar el DataFrame para omitir el año 2014
Internet_ingresos_filtered = Internet_ingresos[Internet_ingresos['Año'] != 2014]

# Ordenar el DataFrame filtrado por Crecimiento_anual de forma decreciente
Internet_ingresos_sorted = Internet_ingresos_filtered.sort_values('Crecimiento_anual', ascending=False)

sns.set(style='darkgrid', palette='viridis')

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
sns.barplot(x='Año', y='Crecimiento_anual', data=Internet_ingresos_sorted)

# Personalizar el gráfico
plt.title('Crecimiento anual en ganancias expresada en porcentaje')
plt.xlabel('Año')
plt.ylabel('Crecimiento anual')

# Mostrar el gráfico
plt.show()

#En este grafico se observa que el mayor crecimiento anual en cuando a ganancias con relacion al año anterior
#está representado por el año 2018, seguido del 2017

Busco el KPI asociado a ganancias

In [ ]:
from sklearn.linear_model import LinearRegression

# Crear una variable para considerar datos hasta 2022, utilizando el método dropna para omitir 2014
datos_hasta_2022 = Internet_ingresos['Crecimiento_anual'].dropna()

# Crear el modelo de regresión lineal asignando las variables independiente y dependiente
X = Internet_ingresos.loc[datos_hasta_2022.index, 'Año'].values.reshape(-1, 1)  # Variable independiente
y = datos_hasta_2022  # Variable dependiente

modelo = LinearRegression()
modelo.fit(X, y)

# Predecir los ingresos para el cuarto trimestre de 2022 y el año 2023
ingresos_2022 = modelo.predict([[2022]])
ingresos_2023 = modelo.predict([[2023]])

# Mostrar los resultados
print("Ingresos proyectados para el cuarto trimestre de 2022:", ingresos_2022[0])
print("Ingresos proyectados para el año 2023:", ingresos_2023[0])


In [ ]:
Internet_ingresos.columns

Se obtien el KPI en forma visual, ganancias esperadas para 4 trimestre del 2022 y 2023

In [ ]:
# Crear el gráfico de crecimiento en ganancias, se realizaz con ingreso en miles por temas de practicidad
plt.plot(Internet_ingresos['Año'], Internet_ingresos['Ingresos (miles de pesos)'], marker='o', label='Crecimiento en ganancias')

# Establecer las metas u objetivos para el tercer trimestre de 2022 y el año 2023
plt.axvline(x=2022.75, color='red', linestyle='--', label='Objetivo T4 2022')
plt.axhline(y=ingresos_2022[0], color='red', linestyle='--')
plt.text(2022.8, ingresos_2022[0]+10000, f'${ingresos_2022[0]:.2f}k')

plt.axvline(x=2023, color='green', linestyle='--', label='Objetivo 2023')
plt.axhline(y=ingresos_2023[0], color='green', linestyle='--')
plt.text(2023.05, ingresos_2023[0]+10000, f'${ingresos_2023[0]:.2f}k')

# Personalizar el gráfico
plt.title('Crecimiento en ganancias y metas')
plt.xlabel('Año')
plt.ylabel('Ingresos (miles de pesos)')
plt.legend()

# Mostrar el gráfico
plt.show()


Realizamos el análisis de penetrancia o acceso a internet por cada 100 hogares, para ello agrupamos por año y provincia, se determina a 
partir de 2017 para no presentar discordancias con los datos de otros df 

In [ ]:
#Muestro mi DF 
Internet_Penetracion

In [ ]:
# Filtrar y procesar los datos por año a partir de 2017
for year, data_year in Internet_Penetracion[Internet_Penetracion['Año'] >= 2017].groupby('Año'):
    # Eliminar las comas de los valores en la columna 'Accesos por cada 100 hogares'
    data_year['Accesos por cada 100 hogares'] = data_year['Accesos por cada 100 hogares'].str.replace(',', '').astype(float)
    # Calcular el promedio del acceso a Internet por cada 100 hogares para cada provincia y trimestre
    data_avg = data_year.groupby(['Provincia', 'Trimestre'])['Accesos por cada 100 hogares'].mean().reset_index()
    # Obtener la provincia con el menor acceso a Internet
    provincia_menor_acceso = data_avg.loc[data_avg['Accesos por cada 100 hogares'].idxmin()]['Provincia']
    # Obtener el valor del menor acceso a Internet
    menor_acceso = data_avg.loc[data_avg['Accesos por cada 100 hogares'].idxmin()]['Accesos por cada 100 hogares']
    # Imprimir el resultado para cada año
    print(f"Año {year}: Provincia con menor acceso a Internet: {provincia_menor_acceso}, Accesos por cada 100 hogares: {menor_acceso}")

Se decide obtener datos de KPI para ello realizo una vez más modelos de regresión considerando crecimiento trimestral y Acceso por cada 100 hogares

In [ ]:
# Primero ordenamos los datos por trimestre, dado que existía un error en la correlación.
datos_ordenados = Internet_Penetracion.sort_values('Trimestre')

# Limpiar y convertir la columna 'Accesos por cada 100 hogares'
datos_ordenados['Accesos por cada 100 hogares'] = datos_ordenados['Accesos por cada 100 hogares'].str.replace(',', '.').astype(float)

# Asignar los datos de entrenamiento
X_train = datos_ordenados['Trimestre'].values.reshape(-1, 1)  # Variable independiente
y_train = datos_ordenados['Accesos por cada 100 hogares']  # Variable dependiente

# Ajustar el modelo utilizando los datos de entrenamiento
modelo.fit(X_train, y_train)

# Realizar la proyección para el cuarto trimestre del 2022
trimestre_2022 = 4
X_2022 = [[trimestre_2022]]
proyeccion_2022 = modelo.predict(X_2022)

# Redondear la proyección a un número entero
proyeccion_2022_entero = int(round(proyeccion_2022[0]))

# Obtener los últimos dos valores de la serie
ultimo_valor = y_train.iloc[-1]
penultimo_valor = y_train.iloc[-2]

# Calcular el crecimiento anterior y promedio
crecimiento_anterior = ultimo_valor - penultimo_valor
crecimiento_promedio = y_train.mean()

# Mostrar el resultado de la proyección y los crecimientos
print("Basado en el crecimiento por cada 100 hogares por trimestre, se espera que para el 4to trimestre del 2022 existan", proyeccion_2022_entero, "accesos por cada 100 hogares.")
print("Crecimiento anterior: {:.2f}".format(crecimiento_anterior))
print("Crecimiento promedio: {:.2f}".format(crecimiento_promedio))



Dado los resultados se define generar la proyección gráfica del KPI

In [ ]:
# Crear un array con los valores de trimestre para la proyección
trimestres_proyeccion = np.array(range(1, trimestre_2022+1))

# Calcular los valores proyectados para cada trimestre
proyeccion_trimestres = modelo.predict(trimestres_proyeccion.reshape(-1, 1))

# Crear el gráfico de línea
plt.figure(figsize=(10, 6))
plt.plot(X_train, y_train, 'o-', label='Datos reales')
plt.plot(trimestres_proyeccion, proyeccion_trimestres, 'r--', label='Proyección')

# Personalizar el gráfico
plt.title('Proyección de accesos por cada 100 hogares')
plt.xlabel('Trimestre')
plt.ylabel('Accesos por cada 100 hogares')
plt.legend()

# Mostrar el gráfico
plt.show()



Iniciamos con el calculo del PBI de Argentina

In [ ]:
import pandas as pd

ruta_archivo = 'Datasets\MetadatosPBI.xlsx'
df_PBI = pd.read_excel(ruta_archivo)

In [ ]:
df_PBI.columns

In [ ]:
df_PBIArgentina = df_PBI[df_PBI['Country Name'] == 'Argentina'].copy()
df_PBIArgentina.drop(['Country Code', 'Indicator Name', 'Indicator Code'], axis=1, inplace=True)


In [ ]:
df_PBIArgentina.columns

In [ ]:
df_PBIArgentina.dtypes


In [ ]:
# Crear una lista de años
años = ['1961', '1962', '1963', '1964', '1965', '1966', '1967',
       '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976',
       '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985',
       '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

# Crear un nuevo DataFrame para almacenar el crecimiento del PBI
df_crecimiento_pbi = pd.DataFrame(index=df_PBIArgentina.index)

# Calcular el crecimiento porcentual del PBI año a año y agregar las columnas al nuevo DataFrame
for i in range(1, len(años)):
    crecimiento_pbi = ((df_PBIArgentina.loc[:, años[i]] - df_PBIArgentina.loc[:, años[i-1]]) / df_PBIArgentina.loc[:, años[i-1]]) * 100
    df_crecimiento_pbi[f'Crecimiento PBI {años[i]}'] = crecimiento_pbi

# Imprimir el DataFrame con los datos de crecimiento del PBI
df_crecimiento_pbi.head(59)



In [ ]:
#Este codigo es solo para que me muestre la cantidad máxima de columnas
pd.set_option('display.max_columns', None)
df_crecimiento_pbi

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Señalo los valores de X
años = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

crecimiento_pbi = [-382.738597, -145.539405, -290.840772, -156.514858, -304.910732, -17.549021, 264.114608, -186.539468]

# Establecer el estilo de Seaborn
sns.set(style="darkgrid")

# Crear el gráfico de líneas
plt.figure(figsize=(10, 6))
sns.lineplot(x=años, y=crecimiento_pbi)

plt.title('Crecimiento Porcentual del Producto Interno Bruto')
plt.xlabel('Año')
plt.ylabel('Crecimiento Porcentual')


Calculamos la correlación de Pearson para el caso entre PBI y Velocidad de Internet Bajada con la finalidad de obtener metricas que indiquen
que el sector de telecomunicaciones si contribuye con el crecimiento del pais. En nuestro caso particular se demuestra una correlacion moderada

In [ ]:
import scipy.stats as stats

df_correlacion = pd.DataFrame({
    'Año': ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'],
    'Crecimiento Porcentual del Producto Interno Bruto': [-382.738597, -145.539405, -290.840772, -156.514858, -304.910732, -17.549021, 264.114608, -186.539468],
    'Ganancias Porcentuales del Sector de Telecomunicaciones': [46.951834, 33.610605, 47.657807, 50.079740, 45.739205, 47.034826, 28.057995, 11.994931]
})

correlation, p_value = stats.pearsonr(df_correlacion['Crecimiento Porcentual del Producto Interno Bruto'], df_correlacion['Ganancias Porcentuales del Sector de Telecomunicaciones'])

print("La correlación de Pearson entre el Crecimiento del Producto Interno Bruto y Ganancias Porcentuales:", correlation)
print("Valor p entre entre el Crecimiento del Producto Interno Bruto y Ganancias Porcentuales:", p_value)

#los resultados indican una correlación negativa débil entre el crecimiento del PIB y las ganancias porcentuales del sector de telecomunicaciones. 
# Sin embargo, debido al valor p mayor que 0.05, no hay suficiente evidencia estadística para respaldar esta correlación como significativa.

Considerando que el resultado demuestra que existe una aparente relacion negativa o inversa sin suficiente evidencia
estadística se decide realizar un modelo de regresión lineal

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

df_correlacion = pd.DataFrame({
    'Año': ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'],
    'Crecimiento Porcentual del Producto Interno Bruto': [-382.738597, -145.539405, -290.840772, -156.514858, -304.910732, -17.549021, 264.114608, -186.539468],
    'Ganancias Porcentuales del Sector de Telecomunicaciones': [46.951834, 33.610605, 47.657807, 50.079740, 45.739205, 47.034826, 28.057995, 11.994931]
})

#Se llama al modelo
modelo = LinearRegression()

# Separar las variables independientes (X) y la variable objetivo (y)
X = df_correlacion[['Crecimiento Porcentual del Producto Interno Bruto']]
y = df_correlacion['Ganancias Porcentuales del Sector de Telecomunicaciones']

# Entrenar el modelo
modelo.fit(X, y)

# Realizar las predicciones
predicciones = modelo.predict(X)

# Calcular el Error Cuadrático Medio (MSE) y el Coeficiente de Determinación (R-squared)
mse = mean_squared_error(y, predicciones)
r2 = r2_score(y, predicciones)

# Imprimir las predicciones, el MSE y el R-squared
print('Predicciones:', predicciones)
print('Error Cuadrático Medio (MSE):', mse)
print('Coeficiente de Determinación (R-squared):', r2)

#Los resultados indican que el modelo de regresión lineal utilizado tiene un ajuste limitado a los datos 
# y no puede explicar de manera adecuada la relación entre el crecimiento del PIB y las ganancias del sector de telecomunicaciones. 
# Es posible que haya otros factores o variables que influyan en las ganancias del sector y que no se están considerando en este análisis univariado